In [83]:
import cv2 as cv  
import numpy as np  

video = cv.VideoCapture('fire.mp4') 
mosaic_size = 10

# 모자이크 적용 함수
def apply_mosaic(frame, mask, mosaic_size):
    for i in range(0, frame.shape[0] - mosaic_size, mosaic_size):      # 세로 방향으로 mosaic_size만큼 증가하며 반복
        for j in range(0, frame.shape[1] - mosaic_size, mosaic_size):  # 가로 방향으로 mosaic_size만큼 증가하며 반복
            if np.any(mask[i:i+mosaic_size, j:j+mosaic_size]):         # 마스크가 적용된 영역이면
                face_region = frame[i:i+mosaic_size, j:j+mosaic_size]  # 모자이크 처리할 영역을 선택
                face_region = cv.resize(face_region, (1, 1), interpolation=cv.INTER_AREA)  # 선택한 영역을 1x1 크기로 축소
                face_region = cv.resize(face_region, (mosaic_size, mosaic_size), interpolation=cv.INTER_NEAREST)  # 다시 원래 크기로 확대
                frame[i:i+mosaic_size, j:j+mosaic_size] = face_region  # 원래 위치에 모자이크 처리된 영역을 덮어쓰기


# video의 모든 frame에 대해 처리
while True:
    finish, frame = video.read()

    # frame 읽기에 실패하면 반복문 종료
    if not finish:
        break

    # frame을 BGR에서 HSV(Hue, Saturation, Value) 색공간으로 변환
    # 특정 색상을 뽑아낼 때는 RGB를 이용하는 것보다, HSV에서 Hue로 색상의 범위를 조정해주는 것이 더 유리
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)  

    # 빨간색 감지를 위한 HSV(Hue, Saturation, Value) 범위 정의
    # OpenCV에서 Hue는 0에서 180의 범위를 가지며 원형으로 이루어져 있기에 두 범위로 나눠서 표현
    red_lo1 = np.array([0, 50, 50])        # 빨간색의 하한 HSV 값
    red_hi1 = np.array([10, 255, 255])     # 빨간색의 상한 HSV 값
    red_lo2 = np.array([170, 50, 50])      # 빨간색의 다른 범위 하한
    red_hi2 = np.array([180, 255, 255])    # 빨간색의 다른 범위 상한
    

    # 빨간색 범위에 해당하는 mask 생성
    mask1 = cv.inRange(hsv, red_lo1, red_hi1)    # 첫 번째 빨간색 범위에 대한 mask
    mask2 = cv.inRange(hsv, red_lo2, red_hi2)    # 두 번째 빨간색 범위에 대한 mask
    mask_full = mask1 + mask2                    # 두 mask를 합쳐서 전체 mask 생성

    hsv[mask_full > 0, 1] = 0                    # mask된 영역의 saturation을 0으로 설정하여 gray tone으로 변경

    frame = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)   # HSV 이미지를 BGR 이미지로 다시 변환

    apply_mosaic(frame, mask_full, mosaic_size)  # 모자이크 효과 적용

    # 결과 이미지 표시
    cv.imshow("result.png", frame)               

    # 25ms 동안 키 입력 대기, ESC가 입력되면 반복문 종료
    key = cv.waitKey(25)                       
    if key == 27:                                
        break  

video.release()         # 비디오 파일 리소스 해제
cv.destroyAllWindows()  # 모든 OpenCV 창 닫기


